# 卷积神经网络

图像处理是神经网络的重要应用场景，同时也是相对于其他机器学习算法最成功的的领域。这其中，**卷积神经网络**（**convolutional neural networks**）在图像处理中扮演者重要的作用。

图像，包括一些特殊的文本等数据都可以看成是一些多维的格点，比如一个文本序列是一维格点组成的；一张黑白图像是二维格点组成的；而一张彩色图像可以看成是一个三维的格点（红、绿、蓝）组成。

在这类数据中，格点之间的空间依赖性是非常重要的。比如如果我们需要识别出图像中的轮廓，那么必须比较相邻像素点之间的色彩亮度对比才能达到。此外，特别是在图像数据中，由于现实中图像存在拍摄角度等方面的差异，所以非常需要一些变化不变性（translation invariant），比如如果将图像进行旋转、平移、放缩等，都应该具有同样的输出结果。

为此我们需要首先介绍卷积的概念。

## 卷积

在数学中，卷积即一个加权滑动平均的操作：$$s\left(t\right)=\int x\left(a\right) w\left(t-a\right)da=\left(x*w\right)\left(t\right)$$或者离散情况下，可以用求和的形式：$$s\left(t\right)=\left(x*w\right)\left(t\right)=\sum_{a=-\infty}^{\infty}x\left(a\right)w\left(t-a\right)$$

以上是一维的卷积，而对于图像而言更常见的是二维卷积：$$s\left(i,j\right)=\left(I*K\right)\left(i,j\right)=\sum_{m}\sum_{n}I\left(m,n\right)K\left(i-m,j-n\right)$$其中$I$为图像，$K$为一个二维的“**核**”（**kernel**），或称**过滤器**（**filter**）。

或者，我们可以将其写成：$$s\left(i,j\right)=\left(K*I\right)\left(i,j\right)=\sum_{m}\sum_{n}I\left(i-m,j-n\right)K\left(m,n\right)$$

以上卷积操作如下图所示：

![](pic/convolution.png "卷积的计算")

卷积这一操作有很多优良的性质。比如我们考虑如下一维的卷积操作：

![](pic/convolution_sharing.png "一维的卷积")

可以看到，这一操作首先是稀疏的，即每个输入只影响很少的输出。而不同的输出之间具有共享参数（parameter sharing）的性质，这也是卷积操作之所以成功的一个关键因素，我们可以认为这种共享参数即在图像的所有位置都进行相同的计算，从而大大降低了参数的维数。而这种操作也使得如果将图像进行平移等操作后其结果是不变的。

下图给出了一个完整的卷积操作的例子：

![](pic/convolution_example.png "完整的卷积操作")

为了理解卷积这一操作的有用性，下图展示了一个卷积操作识别图像的水平边缘的一个例子：

![](pic/convolution_edge.png "识别边缘的卷积")

实际应用中我们可能将卷积操作重复多次，即很多层（layer）：

![](pic/convolution_2layers.png "两层一维卷积")

如此嵌套多次，我们就可以将整个图像遍历，仅仅使用一个不大的filter即可。

此外，在每一层也会使用不止一个filter，而是很多的filter在同时工作。这些不同的filter可能发挥着不同的作用：有的识别图像边缘，有的识别纹理等等。

现在，如果$q$层的输入为$L_q\times B_q \times d_q$的张量，其中$L_q$为高、$B_q$为宽、$d_q$为深度（如黑白=1，彩色=3），如果使用$p$记为其使用的filter（维数为$F_q\times F_q \times d_q$，$p$也被称为channel），那么卷积操作可以记为$$h_{ijp}^{q+1}=b_p +\sum_{r=1}^{F_{q}}\sum_{s=1}^{F_{q}}\sum_{k=1}^{d_{q}}W_{rsk}^{\left(p,q\right)}I_{i+r-1,j+s-1,k}^{q}$$其中$W_{rsk}^{\left(p,q\right)}$为filter，$b_p$是一个额外添加的参数，称为bias。

如此计算，那么第$q+1$层，张量的维数应该为$\left(L_q-F_q+1\right)\times \left(B_q-F_q+1\right) \times d_q$。如此，使用了卷积操作后张量的维数变小了，为了解决这一问题，经常会使用一个称为“padding”的操作，即将图像的外围用0来填充：

![](pic/padding.png "Padding")

在PyTorch中，可以使用torch.nn中的Conv1d、Conv2d、Conv3d进行卷积运算。其基本用法为：
```python
torch.nn.Conv1d(in_channels, out_channels, kernel_size)
```
其中：
  - in_channels：输入的channel数
  - out_channels：输出的channel数
  - kernel_size：kernel或者filter的大小

经过初始化后的模型接受的输入应该为（batch size * channels * shape），其中shape在一维中即一个数字，二维图像则是图像的高和宽，等等。

如下代码所示一维的：

In [1]:
import torch
from torch import nn

I = torch.randn(2, 1, 30)  ## 两条(batch size=2)一维的数组，长度30
m = nn.Conv1d(1, 2, 3)  ## 输入为1个channel，输出为2个channel，kernel size=3
print(I)
m(I)

tensor([[[ 1.6434e+00, -2.8230e-01, -2.1313e-01,  7.2754e-01, -5.5354e-01,
           5.9057e-01,  9.8807e-01, -1.4075e+00, -4.2616e-01,  6.5722e-02,
          -5.7614e-01,  1.3395e+00, -7.3661e-01,  1.3621e+00,  9.8851e-02,
          -2.1018e-03, -2.4750e+00,  9.2634e-01,  9.9607e-01, -1.5778e+00,
          -3.2924e-01,  8.0681e-01, -8.1951e-01, -2.4408e-01, -5.6735e-01,
           8.9273e-01, -1.0219e+00,  2.0591e+00,  3.6543e-01, -2.5302e-01]],

        [[-6.4423e-02,  4.4738e-01, -6.2104e-01,  1.4232e+00,  5.0837e-01,
           2.1792e-02,  6.5933e-01,  4.8968e-01, -7.5851e-01,  5.5631e-01,
           9.9822e-01,  6.9611e-01, -1.1332e+00, -1.0759e+00, -1.1863e+00,
          -4.1275e-01, -6.2044e-01,  1.6789e-01, -7.6150e-01,  3.8373e-01,
           1.1941e+00, -1.4040e-01,  1.1844e+00,  7.2343e-01,  4.5842e-01,
          -1.8874e+00, -1.3599e+00, -7.9036e-01, -3.4693e-01,  1.3191e-01]]])


tensor([[[-0.0269,  0.6611,  0.5502,  0.3199,  0.8952,  0.2225, -0.0532,
           0.8109,  0.5073,  0.6255,  0.7020,  0.2687,  0.8966,  0.1402,
           0.0201,  0.2964,  1.4545,  0.1043, -0.0626,  1.0020,  0.5450,
           0.1120,  0.5685,  0.6294,  0.5845,  0.4702,  1.1201, -0.0509],
         [-0.3771, -0.6389,  0.1835, -0.7376, -0.6610,  0.5869, -0.2897,
          -0.1311,  0.1528, -1.0747,  0.4050, -1.2807, -0.0453, -0.4163,
           1.1524, -1.0397, -0.3638,  0.6384, -0.3646, -0.5182,  0.3594,
          -0.3034,  0.1678, -0.7771,  0.5168, -1.6468, -0.0808, -0.3403]],

        [[ 0.4584,  0.5328,  0.9477,  0.1872,  0.4732,  0.6638,  0.2481,
           0.3474,  0.9465,  0.6077,  0.1212, -0.0520,  0.4216,  0.5282,
           0.6320,  0.5374,  0.5405,  0.4015,  0.9573,  0.5327,  0.3616,
           0.8241,  0.3593,  0.0255, -0.1482,  0.6511,  0.6723,  0.6705],
         [ 0.1705, -1.1798, -0.2873, -0.3932, -0.6831, -0.4494,  0.1536,
          -0.7080, -0.6423, -0.6031,  0.3427, 

或者二维的：

In [2]:
m = nn.Conv2d(1, 2, 3,
              padding='same')  ## 输入为1个channel，输出为2个channel，kernel size=3
I = torch.randn(2, 1, 10, 10)  ## batch size=2，1个channel，10*10的图像
print(I)
m(I)

tensor([[[[-0.4784,  1.0185, -0.8259,  0.4930, -1.4482,  1.1654,  0.6748,
           -0.0533,  0.9492,  1.5287],
          [ 0.8339,  1.5310,  0.1231,  1.2650,  0.5546,  1.3268,  1.0710,
           -0.6233,  0.8028, -1.1140],
          [-0.9204, -1.2998,  0.3365,  0.3597, -0.5115,  1.2597,  0.7898,
           -1.0483,  0.2529, -0.1395],
          [ 2.2342,  0.7048, -0.1214,  0.6801,  0.2634,  0.6818,  0.3185,
           -0.2533,  0.3610, -0.4909],
          [-0.7700, -0.0709,  0.6390, -0.3198,  1.1769,  1.0799, -0.0955,
           -0.7287, -0.6067,  1.0895],
          [ 1.3264, -0.7270,  1.0264, -0.8376, -0.1184,  0.8242, -0.8136,
            1.3612,  0.8424,  0.1554],
          [-0.0459, -0.1205,  1.1111, -1.2750, -0.6345, -1.7988, -1.6840,
           -0.2766, -0.3358,  1.2403],
          [-0.8372, -1.4819,  1.7506, -1.4414, -0.7143, -2.1059,  0.5830,
            0.9444,  0.4861,  1.9648],
          [ 0.1647, -1.4379, -0.5463, -1.1681,  0.4803,  0.3680,  2.8416,
            1.6042, -0

tensor([[[[ 0.1181,  0.2709,  0.8719,  0.2450,  0.5911, -0.0605,  0.4546,
            0.5669, -0.6602,  0.5413],
          [-0.1180, -0.1354, -0.1393, -0.3109,  0.6537, -0.1926,  0.4849,
            0.7192,  0.0431,  0.3363],
          [ 0.5007,  0.5568,  0.4246,  0.2183,  0.4709,  0.0288,  0.4607,
            0.6015, -0.4246,  0.3945],
          [-0.2180,  0.3934, -0.0120,  0.3472,  0.2645,  0.3617,  0.2911,
            0.1720,  0.1272, -0.0862],
          [-0.0591,  0.5360, -0.1902,  0.4084, -0.1774,  0.0263,  0.9233,
            0.0807,  0.1064,  0.2545],
          [ 0.2010,  0.4278, -0.3089,  0.9235, -0.6193, -0.0928, -0.4737,
           -0.8133,  0.5822, -0.0561],
          [-0.1698,  0.3237, -0.7689,  1.0712, -0.2539,  0.3705, -0.2637,
            0.1872,  0.4348, -0.0363],
          [ 0.1820,  0.0682, -0.7244,  0.5819, -0.2806,  0.3811, -0.3048,
            0.6655,  0.5373,  0.0159],
          [ 0.2215,  0.5510, -0.1067, -0.2663, -0.0196, -0.8317, -0.1233,
            0.2416,  0

其中
```python
padding='same'
```
代表进行padding，输出的大小等于输入的大小。

当然，也可以使用sequential组合起来：

In [3]:
m = nn.Sequential(nn.Conv1d(1, 5, 3), nn.Conv1d(5, 5, 3))
I = torch.randn(1, 1, 10)
print(I)
m(I)

tensor([[[ 0.8152, -0.3311, -0.3354, -1.0295, -1.0512, -0.1202,  1.2073,
           0.5802, -1.4066, -0.6785]]])


tensor([[[-1.1578e-03,  9.6408e-02,  1.7319e-01, -2.3793e-01, -4.7724e-01,
           1.3941e-01],
         [-7.2090e-01, -2.9762e-01,  5.0189e-02, -2.3192e-04, -5.1155e-01,
          -6.5965e-01],
         [-2.8672e-01, -2.4596e-01, -1.5633e-01,  2.3501e-01,  3.9131e-01,
          -2.1878e-01],
         [ 1.5520e-01,  3.2128e-01,  4.6064e-01,  3.9710e-01,  7.6769e-02,
           2.4908e-01],
         [-4.4343e-01, -4.3334e-01, -5.4944e-02,  2.2276e-01,  7.0343e-02,
          -2.3771e-01]]], grad_fn=<ConvolutionBackward0>)

此外，在计算卷积时也可以不逐格进行，而是有跳跃的进行，这可以通过“步长”（strides）进行控制。比如，上面介绍的卷积操作stride=1，而如果设定stride=2，那么将会隔一个算一次卷积。

In [4]:
m = nn.Conv1d(1, 5, 3, stride=2)
I = torch.randn(1, 1, 10)
print(I)
m(I)

tensor([[[ 0.7720, -0.1727,  0.9556,  1.5400, -0.6630,  0.0556,  0.9393,
           1.1046, -1.1715, -0.7452]]])


tensor([[[ 0.6747, -0.6034,  1.0351, -0.6779],
         [-0.8716,  0.1063, -0.7748,  0.1150],
         [-0.3573, -0.0988, -0.0408, -0.0222],
         [ 0.3763,  0.6188,  1.0197,  0.2858],
         [ 0.3372,  0.5405, -0.1363,  0.4194]]],
       grad_fn=<ConvolutionBackward0>)

一般而言对于图像问题，经常会使张量的宽和高相等的图像，即$L_q=B_q$，stride一般设置为1或者顶多为2，kernel的大小一般设置为3或者5，而每一层中filter的个数一般取为$2^\#$，即2的幂次方。

## 探测层

卷积仍然是线性函数，为了给模型中带来非线性性，通常在进行几次卷积计算之后，可以使用激活函数为模型添加非线性性，这一阶段被称为detector stage。比如，可以使用ReLU激活函数：

In [5]:
m = nn.Sequential(nn.Conv1d(1, 5, 3), nn.Conv1d(5, 5, 3), nn.ReLU())
I = torch.randn(1, 1, 10)
print(I)
m(I)

tensor([[[ 0.4070,  0.9882, -0.2379,  1.5462, -0.7323, -0.6717, -0.4272,
          -1.6041, -1.2196,  0.1237]]])


tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3007, 0.4009, 0.0000, 0.2578, 0.1454, 0.0000],
         [0.2726, 0.2310, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0930, 0.3140, 0.0000, 0.0925, 0.1053, 0.0000],
         [0.0385, 0.1614, 0.0000, 0.0000, 0.0000, 0.0000]]],
       grad_fn=<ReluBackward0>)

## 池化层

最后一个重要的操作被称为“**池化**”（**pooling**）。池化层，顾名思义，就是将相邻的像素点混合在一起得到一个数字。该计算与卷积层的计算结构类似，不过并不是计算相邻像素点的线性组合，而是将filter内的所有数字计算最大值，也被称为**最大池化**（**max pooling**）。比如下图：

![](pic/max_pooling.png "最大池化")

通过求最大值，计算过程就不太容易收到一些轻微扰动的干扰，从而能达到更好的变化不变性。当然，也有其他的池化方法，比如求均值、L2范数等等，不过应用最广的还是最大池化。

池化同样需要选取filter的大小和步长，比如，可以使用$2\times 2$的filter以及stride=2，此时池化时彼此就不会重叠。不过也有建议池化时应该稍微有所重叠，比如使用$3\times 3$的filter以及stride=2就可以达到一定的重叠效果。

在PyTorch中，可以使用nn.MaxPool1d、nn.MaxPool2d等进行池化操作：

In [6]:
m = nn.Sequential(nn.Conv1d(1, 5, 3, padding='same'),
                  nn.Conv1d(5, 5, 3, padding='same'), nn.ReLU(),
                  nn.MaxPool1d(3, stride=2))
I = torch.randn(1, 1, 10)
print(I)
m(I)

tensor([[[-3.0130,  0.9684,  0.4818, -0.0880,  2.1395, -0.1403, -0.7698,
          -0.3773, -0.9631, -2.0439]]])


tensor([[[0.4623, 0.7707, 0.1496, 0.0000],
         [0.8013, 0.6278, 0.4597, 0.4535],
         [0.5547, 0.0000, 0.5045, 0.6666],
         [0.5640, 0.5640, 0.8361, 0.4466],
         [1.1072, 0.0593, 0.8602, 1.2361]]], grad_fn=<SqueezeBackward1>)

## 全连接层

经过池化所得到的张量可以用于下一步的计算，比如通过一些全连接层继续进行分类问题的建模。

In [7]:
m = nn.Sequential(nn.Conv2d(1, 5, 3, padding='same'),
                  nn.Conv2d(5, 5, 3, padding='same'), nn.ReLU(),
                  nn.MaxPool2d(3, stride=2))
I = torch.randn(1, 1, 10, 10)
x = m(I).view(-1, 5 * 4 * 4)  ## 改变tensor的形状
l = nn.Linear(5 * 4 * 4, 8)
print(I)
print(m(I))
print(x)
l(x)

tensor([[[[-0.0294, -0.4685, -0.3059,  0.9050, -0.8975, -0.5479,  0.0938,
           -0.3804,  0.4278, -0.4231],
          [-1.5257, -0.1318, -0.3209, -1.2235,  1.4738, -0.8019, -0.2671,
           -0.9380, -0.4776, -1.5285],
          [ 0.9853,  0.8027, -0.8442, -0.5961, -0.8095,  0.4834, -1.4699,
            1.4119, -0.2968,  0.5083],
          [ 1.0102, -1.6036, -0.3897, -0.3924, -0.4680,  1.4233, -0.8914,
           -0.7349, -0.4479, -0.5357],
          [-0.5032,  2.0094,  0.3476,  0.0840, -1.0287, -0.0972,  1.0332,
           -1.8118,  0.2056,  0.6455],
          [-0.1289,  0.1916,  0.6270,  0.3029, -2.2327, -0.4496, -0.4181,
            1.5213,  1.0702, -1.1202],
          [-1.7952, -2.1360, -1.3411, -0.4205, -0.1455,  0.0394, -0.3810,
           -0.3734,  0.2495, -1.3439],
          [ 0.1247, -1.5009,  1.1508, -0.5978,  0.7507,  1.2684, -1.2131,
           -1.0102, -0.6482,  0.1112],
          [-0.9798,  1.7430, -0.1506, -0.7951,  0.5934, -1.2594,  0.3977,
           -1.1871, -0

tensor([[-0.0968,  0.2070,  0.1272,  0.0789,  0.3959, -0.1982,  0.3437,  0.2669]],
       grad_fn=<AddmmBackward0>)

# 卷积神经网络在经济学中的应用

在经济学的研究中，卫星图像数据经常被用于经济活跃程度的度量。然而以往基于简单的夜间亮度等信息的度量往往有较大误差。Khachiyan等人（2022）使用CNN模型，利用白天的卫星图像对GDP进行了预测。

他们所使用的卫星图像数据来源于USGS，有七个频段的数据（可见光、近红外、中红外、热成像等），他们将这些图像数据进行网格化（2.4km和1.2km两种），然后将普查数据插补进这些网格中，从而得到每个格点的收入和人口数等数据（作为label）。

由于数据量巨大，我们在这里不再提供完整数据，其完整代码主页：https://github.com/thomas9t/spatial-econ-cnn ，此外完整数据主页：https://drive.google.com/drive/folders/1UgE9rTIyKP_q8Ujfad3bPLdQvz2Z3Xx0 

为了处理图像数据，他们首先定义了卷积层：

```python
from tensorboard.plugins.hparams import api as hp
from utils import *


def conv_block(inputs, n_filter, regularizer, common_args):
    x = tf.keras.layers.Conv2D(filters=n_filter,
                               kernel_size=3,
                               strides=1,
                               padding='same',
                               activation='relu',
                               kernel_regularizer=regularizer,
                               **common_args)(inputs)
    x = tf.keras.layers.Conv2D(filters=n_filter,
                               kernel_size=3,
                               strides=1,
                               padding='same',
                               activation='relu',
                               kernel_regularizer=regularizer,
                               **common_args)(x)
    x = tf.keras.layers.Conv2D(filters=n_filter,
                               kernel_size=3,
                               strides=1,
                               padding='same',
                               activation='relu',
                               kernel_regularizer=regularizer,
                               **common_args)(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    return x

```

注意其使用的并不是PyTorch而是TensorFlow，不过接口相似。在这里他们定义了三个卷积层，每一层都使用了size=3的kernel和ReLU的激活函数。此外还包括了一个L2的正则化项。

经过卷积层之后，他们使用了全连接层：

```python
def dense_block(inputs, n_filter, regularizer, drop_rate, common_args):
    x = tf.keras.layers.Dense(16 * n_filter,
                              activation='relu',
                              kernel_regularizer=regularizer,
                              **common_args)(inputs)
    x = tf.keras.layers.Dropout(drop_rate)(x)
    x = tf.keras.layers.Dense(16 * n_filter, activation='relu',
                              kernel_regularizer=regularizer,
                              **common_args)(x)
    x = tf.keras.layers.Dropout(drop_rate)(x)
    x = tf.keras.layers.Dense(8 * n_filter, activation='relu',
                              kernel_regularizer=regularizer,
                              **common_args)(x)
    x = tf.keras.layers.Dropout(drop_rate)(x)
    return x
```

在每个全连接层之间都使用了Dropout，此外也使用了L2正则化。

之后就可以定义模型了。文章中他们提供了两种模型：一种是level模型，即直接预测人口、收入等变量；另一种是diff模型，即预测两年之间的变化。

此外需要注意的是，虽然他们有三年的数据，并没有分开三年来做，而是将三年的数据混合在一起做，目的是为了降低过拟合。其具体的模型代码：

```python
def make_level_model(img_size, n_bands, l2, nf, dr, with_feature):
    regularizer = tf.keras.regularizers.l2(l2)
    initializer = tf.keras.initializers.glorot_normal()
    common_args = {"kernel_initializer": initializer}

    inputs = tf.keras.layers.Input(shape=(img_size, img_size, n_bands))
    x = conv_block(inputs, nf, regularizer, common_args)
    x = conv_block(x, nf * 2, regularizer, common_args)
    x = conv_block(x, nf * 4, regularizer, common_args)

    x = tf.keras.layers.Flatten()(x)
    if with_feature:
        inputs_features = tf.keras.Input(shape=(34,))
        x = tf.keras.layers.concatenate([x, inputs_features])
        x = dense_block(x, nf, regularizer, dr, common_args)
        output = tf.keras.layers.Dense(1, **common_args)(x)
        model = tf.keras.Model([inputs, inputs_features], output)
        return model
    x = dense_block(x, nf, regularizer, dr, common_args)
    output = tf.keras.layers.Dense(1, **common_args)(x)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model


def make_diff_model(img_size, n_bands, l2, nf, dr, with_feature, model):
    regularizer = tf.keras.regularizers.l2(l2)
    initializer = tf.keras.initializers.glorot_normal()
    common_args = {"kernel_initializer": initializer}

    inputs1 = tf.keras.layers.Input(shape=(img_size, img_size, n_bands))
    inputs2 = tf.keras.layers.Input(shape=(img_size, img_size, n_bands))

    if with_feature:
        inputs_features = tf.keras.Input(shape=(34,))
        level_model = tf.keras.Model(model.input, outputs=model.get_layer('concatenate').output)
        x1 = level_model((inputs1, inputs_features))
        x2 = level_model((inputs2, inputs_features))
        x1 = tf.keras.layers.Flatten()(x1)
        x2 = tf.keras.layers.Flatten()(x2)
        x = tf.keras.layers.Concatenate()([x1, x2])
        x = dense_block(x, nf, regularizer, dr, common_args)
        output = tf.keras.layers.Dense(1, **common_args)(x)
        diff_model = tf.keras.Model([inputs1, inputs2, inputs_features], outputs=output)
        return diff_model
    level_model = tf.keras.Model(model.input, outputs=model.get_layer('max_pooling2d_2').output)
    x1 = level_model(inputs1)
    x2 = level_model(inputs2)
    x1 = tf.keras.layers.Flatten()(x1)
    x2 = tf.keras.layers.Flatten()(x2)
    x = tf.keras.layers.Concatenate()([x1, x2])
    x = dense_block(x, nf, regularizer, dr, common_args)
    output = tf.keras.layers.Dense(1, **common_args)(x)
    diff_model = tf.keras.Model([inputs1, inputs2], outputs=output)
    return diff_model
```

此外，其中的with_feature代表是否加入期初的一些控制变量。

如此，就定义了一个使用图像预测收入、人口的神经网络模型。

最后，他们将数据分为三部分：训练集、验证集，以及一个额外的测试集，其中训练集用于训练，验证集用于选超参数，而测试集用于最终展示预测效果，其结果如下图所示：
![](pic/cnn_t1.png "模型结果")

以及预测图：
![](pic/cnn_f2.png "预测值与真实值")